You can use this template for problems 1, 2 and 3

In [ ]:
import numpy as np
import time
# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

Recent Version of the Code

In [ ]:
# class ROLLNUMBER_Q1:
#   def __init__(self):
#     self.__num_arms = 6
#     self._counts = np.zeros(self.__num_arms)
#     self._values = np.zeros(self.__num_arms)
#     self.__prev_arm = -1
#     self.__count = 0
#     #self._prev

#   def __compute(self,arm):
#     # kl divergence formulae
#     def kl_divergence(p, q):
#       if q == 0 and p == 0:
#           return 0
#       elif q == 0 and not p == 0:
#           return DIV_MAX
#       elif q == 1 and p == 1:
#           return 0
#       elif q == 1 and not p == 1:
#           return DIV_MAX
#       elif p == 0:
#           return np.log(1/(1-q))
#       elif p == 1:
#           return np.log(1/q)
#       return p * np.log(p/q) + (1-p) * np.log((1-p)/(1-q))

#     DIV_MAX = 10000000
#     precision = 1e-7
#     max_iter = 50
#     n = 0
#     p = self._values[arm]
#     lower_bound = self._values[arm]
#     upper_bound = 1
#     # loop to find the best q, i.e ucb
#     while n<max_iter and upper_bound - lower_bound > precision:
#       q = (lower_bound + upper_bound)/2
#       if kl_divergence(p,q) > np.log(1 + self._count*(np.log(self.count)**2))/self._counts[arm]:
#         upper_bound = q
#       else:
#         lower_bound = q
#       n+=1
#     result = (lower_bound + upper_bound)/2.
#     return result

#   def get_action(self,wicket,runs_scored):

#     self.__count += 1
#     if self.__count > 1:
#       self._counts[self.__prev_arm] += 1
#       self._values[self.__prev_arm] += (2-wicket - self._values[self.__prev_arm])/self._counts[self.__prev_arm]
#     #print(self._prev_arm , self.values[self._prev_arm])

#     # exploration phase
#     if self.__count <= self.__num_arms:
#       self.__prev_arm += 1
#       return self.__prev_arm

#     kcl_ucb = np.zeros(self.__num_arms)
#     for arm in range(self.__num_arms):
#       kcl_ucb[arm] = self.__compute(arm)
#     self.__prev_arm = np.argmax(kcl_ucb)
#     # print(self._prev_arm , self.values[self._prev_arm])
#     #action = np.random.randint(0,6)
#     return self.__prev_arm



####################################################################################


class EE21B093_Q1:
  def __init__(self):
    self.pred_mean = np.zeros(6,dtype = float)
    #self.init_action = [i for i in range(6) for j in range(1)]
    self.Time = np.zeros((6,),dtype = int)
    self.UCB = np.zeros((6,),dtype = float)
    self.prev_action=-1
    self.i=0

  def KL_divergence(self,p,q):
    DIV_MAX = 100000
    if q == 0 and p == 0:
      return 0

    elif q == 0 and not p == 0:

      return DIV_MAX
    elif q == 1 and p == 1:

      return 0

    elif q == 1 and not p == 1:

      return DIV_MAX

    elif p == 0:

      return np.log(1/(1-q))

    elif p == 1:

      return np.log(1/q)

    return p*np.log(p/q) + (1-p)*np.log((1-p)/(1-q))


  def KL_UCB(self,action,time):

    b=1
    bound=np.log(1+time*(np.log(time))**2)/self.Time[action]
    pred_mean=self.pred_mean[action]
    a=pred_mean
    for i in range(10):
      mu=(a+b)/2
      if (self.KL_divergence(pred_mean,mu) < bound):
        a=mu
      else:
        b=mu
    return (a+b)/2

  def get_action(self,wicket,runs_scored):

    reward=1-wicket
    while(self.i<6): #Playing Every Arm once
      if(self.prev_action != -1):
        self.pred_mean[self.prev_action]+=(1-wicket - self.pred_mean[self.prev_action])/self.Time[self.prev_action]
      #ret = self.init_action[self.i]
      ret = self.i
      self.Time[ret] = self.Time[ret] + 1
      self.prev_action = ret
      self.i+=1
      return int(ret)


    self.pred_mean[self.prev_action]+=(1-wicket - self.pred_mean[self.prev_action])/self.Time[self.prev_action]
    #print(self.pred_mean)
    for j in range(6):
      self.UCB[j]=self.KL_UCB(j,self.i)
    ret = np.argmax(self.UCB)
    self.Time[ret] = self.Time[ret] + 1
    self.prev_action = ret
    self.i+=1
    return ret




In [ ]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__regret_w = 0
    self.__regret_s = 0
    self.__wicket = 0
    self.__regret_rho = 0
    self.__p_out = np.array([0.001,0.5,0.12,0.03,0.1,0.3])
    self.__p_run = np.array([1,0.9,0.85,0.8,0.75,0.7])
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__s = (1-self.__p_out)*self.__p_run*self.__action_runs_map
    self.__rho = self.__s/self.__p_out


  def __get_action(self):
    self.__start_time      = time. time()
    action          = self.agent.get_action(self.__wicket,self.__runs_scored)
    self.__end_time        = time. time()
    self.__run_time   = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    pout = self.__p_out[action]
    prun= self.__p_run[action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0

    for ball in range(self.num_balls):
      action = self.__get_action()
      self.__wicket, self.__runs_scored   = self.__get_outcome(action)
      self.__total_runs     = self.__total_runs + self.__runs_scored
      self.__total_wickets  = self.__total_wickets + self.__wicket
      self.__regret_w       = self.__regret_w+ (self.__p_out[action]-np.min(self.__p_out))
      self.__regret_s       = self.__regret_s+ (np.max(self.__s) - self.__s[action])
      self.__regret_rho       = self.__regret_rho+ (np.max(self.__rho)-self.__rho[action])
    return self.__regret_w,self.__regret_s,self.__regret_rho, self.__total_runs, self.__total_wickets, self.__run_time


In [ ]:
agent = EE21B093_Q1()
environment = Environment(100,agent)
regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time = environment.innings()
print("aadharsh",regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time)


aadharsh 1.045 284.08599999999984 7632.233333333346 9 0 0.010392189025878906


In [ ]:
print(regret_w,regret_s,reger_rho,total_runs,total_wickets,run_time)

8.696999999999996 29299.985999995162 775274.0999998612 112 20 2.642613649368286
